In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
from scipy.ndimage import uniform_filter1d, gaussian_filter1d
import pandas as pd
import pymap3d as pm
from pyulog import ULog
from plotly.subplots import make_subplots
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is


In [ ]:
data_num = "02"

ulog_file_name = f'../Data/logs/Data{data_num}.ulg'
ulog_file_name = f'../Data/logs/SoCalShootout15.ulg'

ulog = ULog(ulog_file_name)
data = ulog.data_list

for d in data:
    print(d.name)

imu_data = ulog.get_dataset("imuData").data
gps_data = ulog.get_dataset("dualGPS").data

# match gps data to each other based on msGPS
_, gps1_matched, gps2_matched = np.intersect1d(gps_data['gps1.msGPS'], gps_data['gps2.msGPS'], return_indices=True)

# make a histogram of the time between gps readings
# between 0 and 1 sec
fig = go.Figure()
fig.add_histogram(x=np.diff(gps_data['gps1.msGPS'])/1000, name='histogram', xbins=dict(start=0.0, end=0.5, size=0.02))
fig.add_histogram(x=np.diff(gps_data['gps2.msGPS'])/1000, name='histogram', xbins=dict(start=0.0, end=0.5, size=0.02))
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()
# fig2 = go.Figure()
# fig2.add_trace(go.Scatter(x=gps_data['gps1.msGPS']/1000, y=np.diff(gps_data['gps1.msGPS'])/1000, mode='markers'))
# fig2.add_trace(go.Scatter(x=gps_data['gps2.msGPS']/1000, y=np.diff(gps_data['gps2.msGPS'])/1000, mode='markers'))
# fig2.show()

for key in gps_data.keys():
    if key[:4] == 'gps2':
        gps_data[key] = gps_data[key][gps2_matched]
    else:
        gps_data[key] = gps_data[key][gps1_matched]

print(gps_data['gps1.latitude'].shape)
good_inds = (gps_data['gps1.latitude'] != 0.0) & (gps_data['gps1.longitude'] != 0.0) & (gps_data['gps2.latitude'] != 0.0) & (gps_data['gps2.longitude'] != 0.0)
# # filter out bad data for all gps data
for key in gps_data.keys():
    gps_data[key] = gps_data[key][good_inds]
print(gps_data['gps1.latitude'].shape)


print(imu_data.keys())
print(gps_data.keys())

In [ ]:
imu_data['aX'] = imu_data['aX'] / 9.81
imu_data['aY'] = imu_data['aY'] / 9.81
imu_data['aZ'] = imu_data['aZ'] / 9.81

# swap x and y axes
# tire_data['aX'], tire_data['aY'] = tire_data['aY'], tire_data['aX'].copy()
# tire_data['gX'], tire_data['gY'] = tire_data['gY'], tire_data['gX'].copy()
imu_data['time'] = imu_data['timestamp'] / 1000000
gps_data['time'] = gps_data['timestamp'] / 1000000


In [ ]:
# plot the data
fig = make_subplots(rows=8, cols=1, shared_xaxes=True)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['aX'], name='aX'), row=1, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['aY'], name='aY'), row=1, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['aZ'], name='aZ'), row=1, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['gX'], name='gX'), row=2, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['gY'], name='gY'), row=2, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['gZ'], name='gZ'), row=2, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=np.sqrt(imu_data['gX']**2 + imu_data['gY']**2 + imu_data['gZ']**2), name='gA'), row=2, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=uniform_filter1d(np.sqrt(imu_data['gX']**2 + imu_data['gY']**2 + imu_data['gZ']**2), 20), name='gA'), row=2, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['temp'], name='Temperature'), row=3, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=imu_data['buff'], name='Queue'), row=3, col=1)
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=1000000/imu_data['dt'], name='Hz'), row=4, col=1) # , mode="markers"
# time_dt = np.zeros(len(imu_data['time']))
# time_dt[1:] = np.diff(imu_data['time'])
# fig.add_trace(go.Scattergl(x=imu_data['time'], y=1/time_dt, name='Hz Real'), row=4, col=1)

# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps1.latitude'], name=f'gps_data_{1}'), row=5, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps1.longitude'], name=f'gps_data_{1}'), row=6, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps1.altitude'], name=f'gps_data_{1}'), row=7, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps1.h_accuracy'], name=f'gps_data_{1} horizontal'), row=8, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps1.v_accuracy'], name=f'gps_data_{1} vert'), row=8, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps2.latitude'], name=f'gps_data_{2}'), row=5, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps2.longitude'], name=f'gps_data_{2}'), row=6, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps2.altitude'], name=f'gps_data_{2}'), row=7, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps2.h_accuracy'], name=f'gps_data_{2} horizontal'), row=8, col=1)
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps2.v_accuracy'], name=f'gps_data_{2} vert'), row=8, col=1)

fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()


In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
datum = [gps_data['gps1.latitude'].mean(), gps_data['gps1.longitude'].mean()]
fig = px.scatter_mapbox(lat=[datum[0]], lon=[datum[1]], size_max=15, zoom=14)
fig.add_scattermapbox(lat=gps_data['gps1.latitude'], lon=gps_data['gps1.longitude'], name=f'gps_data_{1}')
fig.add_scattermapbox(lat=gps_data['gps2.latitude'], lon=gps_data['gps2.longitude'], name=f'gps_data_{2}')
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000, mapbox_style="mapbox://styles/eppicjr/ck4qx60vr00jy1co6u7ckgg6e") # 
fig.show()

In [ ]:
# Convert to enu from lat/lon
fig = go.Figure()
def convert_gps_to_enu(data_p, i):
    east, north, up = pm.geodetic2enu(data_p[f'gps{i}.latitude'], data_p[f'gps{i}.longitude'], data_p[f'gps{i}.altitude'], datum[0], datum[1], 0)
    fig.add_trace(go.Scattergl(x=east, y=north, mode='markers', marker=dict(color=data_p['time'], colorscale='Viridis', showscale=True), name=f'gps_data_{i}'))
    data_p[f'gps{i}.east'] = east
    data_p[f'gps{i}.north'] = north
    data_p[f'gps{i}.up'] = up
    # calculate heading and speed
    data_p[f'gps{i}.heading'] = np.zeros(len(gps_data['timestamp']))
    data_p[f'gps{i}.speed'] = np.zeros(len(gps_data['timestamp']))
    # vel in enu
    data_p[f'gps{i}.vel_east'] = np.zeros(len(gps_data['timestamp']))
    data_p[f'gps{i}.vel_north'] = np.zeros(len(gps_data['timestamp']))
    data_p[f'gps{i}.vel_up'] = np.zeros(len(gps_data['timestamp']))
    data_p[f'gps{i}.vel_east'][1:] = np.diff(data_p[f'gps{i}.east'])/np.diff(gps_data[f'gps{i}.msGPS'])*1000
    data_p[f'gps{i}.vel_north'][1:] = np.diff(data_p[f'gps{i}.north'])/np.diff(gps_data[f'gps{i}.msGPS'])*1000
    data_p[f'gps{i}.vel_up'][1:] = np.diff(data_p[f'gps{i}.up'])/np.diff(gps_data[f'gps{i}.msGPS'])*1000
    # heading and speed
    data_p[f'gps{i}.heading'][1:] = np.arctan2(data_p[f'gps{i}.vel_east'][1:], data_p[f'gps{i}.vel_north'][1:])
    data_p[f'gps{i}.speed'][1:] = np.sqrt(data_p[f'gps{i}.vel_east'][1:]**2 + data_p[f'gps{i}.vel_north'][1:]**2)
    data_p[f'gps{i}.heading'][0] = data_p[f'gps{i}.heading'][1]
    data_p[f'gps{i}.speed'][0] = data_p[f'gps{i}.speed'][1]

    # filter heading when velocity is below 0.5 m/s, filter the heading as a function of the speed
    speed_threshold = 0.75
    data_p[f'gps{i}.heading_old'] = data_p[f'gps{i}.heading'].copy()
    low_speed = data_p[f'gps{i}.speed'] < speed_threshold
    filter_strength = data_p[f'gps{i}.speed']/speed_threshold
    data_p[f'gps{i}.heading'][1:-1][data_p[f'gps{i}.speed'][1:-1] < 0.5] = np.nan
    data_p[f'gps{i}.heading'] = pd.Series(data_p[f'gps{i}.heading']).interpolate().values
    filtered_heading = uniform_filter1d(data_p[f'gps{i}.heading'], size=20)
    data_p[f'gps{i}.heading'][low_speed] = filtered_heading[low_speed] * (1-filter_strength[low_speed]) + data_p[f'gps{i}.heading'][low_speed] * filter_strength[low_speed]
    data_p[f'gps{i}.heading'] = np.unwrap(data_p[f'gps{i}.heading'])

convert_gps_to_enu(gps_data, 1)
convert_gps_to_enu(gps_data, 2)
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()
    

In [ ]:
# Distance between points
dist = np.sqrt((np.array(gps_data['gps1.east']) - np.array(gps_data['gps2.east']))**2 + (np.array(gps_data['gps1.north']) - np.array(gps_data['gps2.north']))**2)
angle = np.arctan2(np.array(gps_data['gps1.east']) - np.array(gps_data['gps2.east']), np.array(gps_data['gps1.north']) - np.array(gps_data['gps2.north']))
angle = np.unwrap(angle)
def wrap_to_pi(angle):
    return (angle + np.pi) % (2 * np.pi) - np.pi

low_angular_rate_samples = np.interp(gps_data['time'], imu_data['time'], uniform_filter1d(np.sqrt(imu_data['gX']**2 + imu_data['gY']**2 + imu_data['gZ']**2), 20)) < 0.1 # in deg/s
straightline_time = (((gps_data['gps1.speed'] + gps_data['gps2.speed']) / 2) > 0.5) & (low_angular_rate_samples)
heading_offset = np.average(angle[straightline_time] - ((gps_data['gps1.heading'] + gps_data['gps2.heading'])[straightline_time] / 2))
print(f'heading offset: {np.rad2deg(heading_offset)}')
angle = angle - heading_offset
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=dist, name='dist'), row=1, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(angle), name='angle'), row=2, col=1)

fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps1.speed'], name=f'gps_data_{1}_speed'), row=3, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps1.heading']), name=f'gps_data_{1}_heading'), row=2, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps1.carrSoln']), name=f'gps_data_{1}_soln'), row=4, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps1.h_accuracy']), name=f'gps_data_hacc{1}'), row=1, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps1.v_accuracy']), name=f'gps_data_vacc{1}'), row=1, col=1)

fig.add_trace(go.Scattergl(x=gps_data['time'], y=gps_data['gps2.speed'], name=f'gps_data_{2}_speed'), row=3, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps2.heading']), name=f'gps_data_{2}_heading'), row=2, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps2.carrSoln']), name=f'gps_data_{2}_soln'), row=4, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps2.h_accuracy']), name=f'gps_data_hacc{2}'), row=1, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps2.v_accuracy']), name=f'gps_data_vacc{2}'), row=1, col=1)

fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()

In [ ]:
# integrate gyro to get heading
gyro_heading = np.cumsum(np.array(imu_data['gZ'])*np.diff(imu_data['time'], prepend=0))
gyro_heading = np.insert(gyro_heading, 0, 0)
gyro_heading = gyro_heading * -1.0
gyro_heading = np.unwrap(gyro_heading)
# gyro_heading = gyro_heading + angle[0]

deviation = gyro_heading[:-1] - np.interp(imu_data['time'], gps_data['time'], np.unwrap(angle))

corr_heading = gyro_heading[:-1] - uniform_filter1d(deviation, size=1000)

course_offset = np.pi/4
fig = go.Figure()
fig.add_trace(go.Scattergl(x=imu_data['time'], y=np.rad2deg(gyro_heading), name='gyro_heading'))
fig.add_trace(go.Scattergl(x=imu_data['time'], y=np.rad2deg(corr_heading), name='corr_heading'))
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(np.unwrap(angle)), name='angle'))
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(np.unwrap(gps_data['gps1.heading']) + course_offset), name=f'gps_data_{1}_heading'))
fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(np.unwrap(gps_data['gps2.heading']) + course_offset), name=f'gps_data_{2}_heading'))
beta_one = np.interp(imu_data['time'], gps_data['time'], np.unwrap(gps_data['gps1.heading']) + course_offset) - np.unwrap(corr_heading)
beta_two = np.interp(imu_data['time'], gps_data['time'], np.unwrap(gps_data['gps2.heading']) + course_offset) - np.unwrap(corr_heading)
beta_filt_len = 50
beta_one_filt = uniform_filter1d(beta_one, size=beta_filt_len)
beta_two_filt = uniform_filter1d(beta_two, size=beta_filt_len)
beta_avg = (beta_one_filt + beta_two_filt)/2
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps1.heading']), name=f'gps_data_{1}_heading_w'))
# fig.add_trace(go.Scattergl(x=gps_data['time'], y=np.rad2deg(gps_data['gps2.heading']), name=f'gps_data_{2}_heading_w'))
fig.add_trace(go.Scattergl(x=imu_data['time'], y=np.rad2deg(beta_one), name=f'gps_data_{1}_beta'))
fig.add_trace(go.Scattergl(x=imu_data['time'], y=np.rad2deg(beta_two), name=f'gps_data_{2}_beta'))
fig.add_trace(go.Scattergl(x=imu_data['time'], y=np.rad2deg(beta_one_filt), name=f'gps_data_{1}_beta_filt'))
fig.add_trace(go.Scattergl(x=imu_data['time'], y=np.rad2deg(beta_two_filt), name=f'gps_data_{2}_beta_filt'))
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()
fig2 = go.Figure()
# fig2.add_trace(go.Scattergl(x=gps_data['gps1.east'], y=gps_data['gps1.north'], mode='markers', marker=dict(color=np.interp(gps_data['time'], imu_data['time'], np.rad2deg(beta_one_filt)), colorscale='Viridis', showscale=True), name=f'gps_data_{1}'))
# fig2.add_trace(go.Scattergl(x=gps_data['gps2.east'], y=gps_data['gps2.north'], mode='markers', marker=dict(color=np.interp(gps_data['time'], imu_data['time'], np.rad2deg(beta_two_filt)), colorscale='Viridis', showscale=True), name=f'gps_data_{2}'))
cl_east, cl_north, cl_beta = (gps_data['gps1.east'] + gps_data['gps2.east']) / 2, (gps_data['gps1.north'] + gps_data['gps2.north'])/2, np.interp(gps_data['time'], imu_data['time'], beta_avg)
beta_lim_mask = np.abs(cl_beta) < np.deg2rad(60)
# fig2.add_trace(go.Scattergl(x=cl_east[beta_lim_mask], y=cl_north[beta_lim_mask], mode='markers', showlegend=False, marker=dict(color=np.rad2deg(cl_beta[beta_lim_mask]), colorscale='Viridis', showscale=True), name=f'gps_data_avg', hovertext=np.rad2deg(cl_beta[beta_lim_mask]))) # 
# add arrows to show heading and velocity direction
# heading unit vector (x,y) = (cos(theta), sin(theta))
headddd = -np.interp(gps_data['time'], imu_data['time'], np.unwrap(corr_heading)) + np.pi/2 + course_offset
head_unit_vec = 2
# x_head, y_head = np.cos(headddd) * head_unit_vec, np.sin(headddd) * head_unit_vec
vel_unit_vec = (gps_data['gps1.speed'] + gps_data['gps2.speed']) / (2 * 5)
vel_head = ((gps_data['gps1.heading'] + gps_data['gps2.heading']) / -2) + np.pi/2
x_head, y_head = np.cos(vel_head+cl_beta) * head_unit_vec, np.sin(vel_head+cl_beta) * head_unit_vec
x_vel, y_vel = vel_unit_vec * np.cos(vel_head), vel_unit_vec * np.sin(vel_head)
# create a list of arrow start, end, and nan to plot
def create_arrows(x, y, x0, y0):
    arrow_start = np.array([x, y]).T
    arrow_end = np.array([x + x0, y + y0]).T
    arrow_points = np.zeros((arrow_start.shape[0]*3, 2))
    arrow_points[::3, :] = arrow_start
    arrow_points[1::3, :] = arrow_end
    arrow_points[2::3, :] = np.nan
    return arrow_points
vel_arrows = create_arrows(cl_east[beta_lim_mask], cl_north[beta_lim_mask], x_vel[beta_lim_mask], y_vel[beta_lim_mask])
beta_arrows = create_arrows(cl_east[beta_lim_mask], cl_north[beta_lim_mask], x_head[beta_lim_mask], y_head[beta_lim_mask])
fig2.add_trace(go.Scattergl(x=beta_arrows[:, 0], y=beta_arrows[:, 1], mode='lines', line=dict(color='green', width=1), name='Velocity', showlegend=False))
fig2.add_trace(go.Scattergl(x=vel_arrows[:, 0], y=vel_arrows[:, 1], mode='lines', line=dict(color='red', width=1), name='Heading', showlegend=False))
fig2.update_layout(title_text=f"Vehicle Side Slip Angle", height=1000, width=1600) # template="plotly_dark", 
fig2.show()

In [ ]:
def create_arrows_enu(x, y, x0, y0):
    arrow_start = np.array([x, y]).T
    arrow_end = np.array([x0, y0]).T
    arrow_points = np.zeros((arrow_start.shape[0]*3, 2))
    arrow_points[::3, :] = arrow_start
    arrow_points[1::3, :] = arrow_end
    arrow_points[2::3, :] = np.nan
    return arrow_points
cl_lat, cl_lon, cl_alt = pm.enu2geodetic(cl_east, cl_north, np.zeros(cl_north.shape), datum[0], datum[1], 0)
head_lat, head_lon, head_alt = pm.enu2geodetic(cl_east + x_head, cl_north + y_head, np.zeros(cl_north.shape), datum[0], datum[1], 0)
vel_lat, vel_lon, vel_alt = pm.enu2geodetic(cl_east + x_vel, cl_north + y_vel, np.zeros(cl_north.shape), datum[0], datum[1], 0)
beta_arrows_ll = create_arrows_enu(cl_lat[beta_lim_mask], cl_lon[beta_lim_mask], head_lat[beta_lim_mask], head_lon[beta_lim_mask])
vel_arrows_ll = create_arrows_enu(cl_lat[beta_lim_mask], cl_lon[beta_lim_mask], vel_lat[beta_lim_mask], vel_lon[beta_lim_mask])

fig = px.scatter_mapbox(lat=[datum[0]], lon=[datum[1]], size_max=15, zoom=14)
fig.add_scattermapbox(lat=beta_arrows_ll[:, 0], lon=beta_arrows_ll[:, 1], mode='lines', name=f'Heading')
fig.add_scattermapbox(lat=vel_arrows_ll[:, 0], lon=vel_arrows_ll[:, 1], mode='lines', name=f'Vel')
fig.update_layout(height=800, width=1600, mapbox_style="mapbox://styles/eppicjr/ck4qx60vr00jy1co6u7ckgg6e") # template="plotly_dark", title_text=f"Hi", 
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=(gps_data['gps1.msGPS']/1000), name=f'gps_data_{1}'), row=1, col=1)
fig.add_trace(go.Scattergl(x=gps_data['time'], y=(gps_data['gps2.msGPS']/1000), name=f'gps_data_{2}'), row=1, col=1)
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()

In [ ]:
print(gps_data)

In [ ]:
diff = gps_data['gps1.msGPS'].astype(np.int64) - gps_data['gps2.msGPS'].astype(np.int64)
fig = go.Figure()
fig.add_scatter(x=gps_data['time'], y=diff/1000)
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()
print(diff)
print(gps_data['gps1.msGPS'])
print(gps_data['gps2.msGPS'])
print(gps_data['gps1.msGPS'].dtype)